In [1]:
import torch
import numpy as np

In [2]:
data=[[1,2],[3,4]]
x_data=torch.tensor(data) #create the tensor form real data,maybe it like a kind of forced type conversion
np_array=np.array(data)
x_np=torch.from_numpy(np_array) #from a numpy array
x_ones=torch.ones_like(x_data) #from another tensor create an ones tensors that [[1,1,...,111],[11,,1,,1]]
x_rand=torch.rand_like(torch.ones(3,4),dtype=torch.float)#create a tensor whose data is random from another tensor and conver thire dtype

In [3]:
# three type tensors
shape=(2,4)
rand_tensor=torch.rand(shape)
print(rand_tensor)
ones_tensor=torch.ones(shape)
print(ones_tensor)
zeros_tensor=torch.zeros(shape)
print(zeros_tensor)

tensor([[0.9339, 0.9380, 0.6946, 0.0708],
        [0.8964, 0.6306, 0.0063, 0.7667]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [6]:
#to see the attributes of tensor
tensor=torch.rand(3,4)
print(f"Shape of tensor: {tensor.shape}") 
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")
# if torch.cuda.is_available():
#   tensor = tensor.to('cuda')
#   print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [19]:
#tensor operation
tensor=torch.ones(4,4)
tensor[:,1]=0#0:2 means 0 to 1 and use ',' to distinguish different dimensions
tensor[:,-1]=2
print(tensor)

t1=torch.cat([tensor,tensor],dim=0)#dimension means each number in order in tensor's shape
print(t1)
print(t1.shape)
tensor@tensor.T# @ means matrix multiplication and the T means matrix transpose
tensor.add_(5)
tensor

tensor([[1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.]])
tensor([[1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.],
        [1., 0., 1., 2.]])
torch.Size([8, 4])


tensor([[6., 5., 6., 7.],
        [6., 5., 6., 7.],
        [6., 5., 6., 7.],
        [6., 5., 6., 7.]])

In [7]:
#torch.autograd
#train single model
import torch,torchvision#torchvision include many toolkit of image
model=torchvision.models.resnet18(pretrained=True)
data=torch.rand(1,3,64,64)
labels=torch.rand(1,1000)
prediction=model(data)#forward pass
loss=(((prediction-labels)**2).sum(dim=1))#calculate the loss funtion
print(loss)
loss.backward()#call the backward propagation 
#Autograd then calculates and stores the gradients for each model parameter in the parameter’s .grad attribute.
#next we will use the stored gradient to optimize the parameter
#so we need a optimizer
optim=torch.optim.SGD(model.parameters(), lr=1e-2,momentum=0.9)#define the leval of optimizer
optim.zero_grad()
#From the office text:You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.
optim.step()#gradient descent
for i in range(50):
    prediction=model(data)
    loss=(((prediction-labels)**2).sum(dim=1))
    loss.backward()
    optim=torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
    optim.zero_grad()
    optim.step()
prediction=model(data)
loss=((prediction-labels).sum(dim=1))
print(loss)#Why did the loss become larger by several times trains? Beacause the calculateed gradient each time all store at optimizer

tensor([1078.9653], grad_fn=<SumBackward1>)
tensor([-495.1635], grad_fn=<SumBackward1>)


In [2]:
#training an image classifier
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

import torch.nn as nn
import torch.nn.functional as F

#Defien an neural network 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net.to(device)


import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)


#Test the model

dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

net = Net()
net.load_state_dict(torch.load(PATH))

outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))



In [15]:
#To see the corret rate
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 59 %


In [16]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 66.9 %
Accuracy for class: car   is 70.9 %
Accuracy for class: bird  is 48.2 %
Accuracy for class: cat   is 60.2 %
Accuracy for class: deer  is 55.1 %
Accuracy for class: dog   is 27.3 %
Accuracy for class: frog  is 70.7 %
Accuracy for class: horse is 67.1 %
Accuracy for class: ship  is 54.6 %
Accuracy for class: truck is 76.3 %


In [1]:
#Training on GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net.to(device)
inputs, labels = data[0].to(device), data[1].to(device)

NameError: name 'torch' is not defined